In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import pandas as pd
import tensorflow as tf

In [145]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test (1).csv
Saving heart_train.csv to heart_train (1).csv


In [0]:
!rm "heart_test.csv"
!rm "heart_train.csv"
#!rm "heart.h5"

In [0]:
dftrain = pd.read_csv("heart_train.csv")
dftest = pd.read_csv("heart_test.csv")

In [0]:
dftrain['famhist'] = pd.Categorical(dftrain['famhist'])
dftrain['famhist'] = dftrain.famhist.cat.codes
dftest['famhist'] = pd.Categorical(dftest['famhist'])
dftest['famhist'] = dftest.famhist.cat.codes

In [0]:
chdtrain = dftrain.pop('chd')
chdtest = dftest.pop('chd')

In [0]:
datasettrain = tf.data.Dataset.from_tensor_slices((dftrain.values, chdtrain.values))
datasettest = tf.data.Dataset.from_tensor_slices((dftest.values, chdtest.values))

In [0]:
train_dataset = datasettrain.shuffle(len(dftrain)).batch(6)

In [0]:
test_dataset = datasettest.shuffle(len(dftest)).batch(1)

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [270]:
model = get_compiled_model()
model.fit(train_dataset, epochs=6)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/6
66/66 [==============================] - 1s 21ms/step - loss: 3.1997 - accuracy: 0.5025
Epoch 2/6
66/66 [==============================] - 0s 2ms/step - loss: 1.8380 - accuracy: 0.5859
Epoch 3/6
66/66 [==============================] - 0s 2ms/step - loss: 1.5512 - accuracy: 0.6010
Epoch 4/6
66/66 [==============================] - 0s 2ms/step - loss: 1.4137 - accuracy: 0.6035
Epoch 5/6
66/66 [==============================] - 0s 2ms/step - loss: 1.2023 - accuracy: 0.6338
Epoch 6/6
66/66 [=====

In [271]:
model_loss, model_acc = model.evaluate(test_dataset, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

66/66 - 0s - loss: 1.0240 - accuracy: 0.6364
Model Loss:    1.02
Model Accuray: 63.6%
